In [29]:
import cv2
import os
import pandas as pd
import numpy as np

In [38]:
def extractcharacters(plateimage):
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    characters = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 10 < w < 100 and 30 < h < 200:
            char = binary[y:y+h, x:x+w]
            char = cv2.resize(char, (20, 40))  
            characters.append((x, char))
    
    characters = sorted(characters, key=lambda item: item[0])
    return [char for _, char in characters]

In [40]:
def recognizecharacters(characters):
    templates = {
        '0': np.zeros((40, 20)),
        '1': np.zeros((40, 20)),
    }
    recognized = ""
    for char in characters:
        best_match, min_diff = None, float('inf')
        for label, template in templates.items():
            diff = np.sum(np.abs(template - char))
            if diff < min_diff:
                min_diff = diff
                best_match = label
        recognized += best_match or "?"
    return recognized


In [41]:
def processimages(folderpath, coordinatefile, numberfile):
    cascadepath = cv2.data.haarcascades + "haarcascade_plate_number.xml"
    classifier = cv2.CascadeClassifier(cascadepath)
    
    if not os.path.exists(cascadepath):
        print(f"Classifier file not found at {cascadepath}")
        return

    coordinatedata = []
    numberdata = []
    
    for filename in os.listdir(folderpath):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            imagepath = os.path.join(folderpath, filename)
            plates, image = detect_number_plate(imagepath, classifier)
            
            for plate in plates:
                Xmin, Xmax, Ymin, Ymax = plate
                plateimage = image[Ymin:Ymax, Xmin:Xmax]
                characters = extract_characters(plate_image)
                platenumber = recognize_characters(characters)
                coordinate_data.append({
                    "Image": filename,
                    "Xmin": Xmin, "Xmax": Xmax,
                    "Ymin": Ymin, "Ymax": Ymax
                })
                
                number_data.append({
                    "Image": filename,
                    "Plate Number": plate_number
                })

    coordinatedf = pd.DataFrame(coordinatedata)
    coordinatedf.to_excel(coordinatefile, index=False)
    print(f"Coordinates saved to {coordinatefile}")
    
    number_df = pd.DataFrame(number_data)
    number_df.to_excel(number_file, index=False)
    print(f"Number plates saved to {numberfile}")


In [42]:
if __name__ == "__main__":
    images_folder = r"C:\Users\harshit rana\OneDrive\Desktop\numberplateproject\test\test"
    coordinate_excel = r"C:\Users\harshit rana\OneDrive\Desktop\numberplateproject\CordinateOutput\number_plate_coordinates.xlsx"
    number_excel = r"C:\Users\harshit rana\OneDrive\Desktop\numberplateproject\numberplateoutput\number_plate_numbers.xlsx"
    
    if not os.path.exists(images_folder):
        print(f"Folder not found: {images_folder}")
    else:
        process_images(images_folder, coordinate_excel, number_excel)


Coordinates saved to C:\Users\harshit rana\OneDrive\Desktop\numberplateproject\CordinateOutput\number_plate_coordinates.xlsx
Number plates saved to C:\Users\harshit rana\OneDrive\Desktop\numberplateproject\numberplateoutput\number_plate_numbers.xlsx
